# DNN Assignment
In this assignment you are working together with your teammates from the second project. You will apply your new knowledge about dense neural networks to the data from your ML project to investigate, if you can make further improvements on prediction performance. Your data is (hopefully) already cleaned and transformed (this was part of your ML project) such that you can focus fully on feeding it to your neural network. Use TensorFlow 2.x in this assignment as it makes training with real-life data much more easier with many implemented features (e.g. early-stopping, TensorBoard, regularization, etc.). 

In this notebook you will learn
- how to apply a neural network to your own data using TensorFlow 2.x
- how to tune the network and monitor learning
- how to train several networks and ensemble them into a stronger model

# Module loading
Load all the necessary packages for your assignment. We give you some modules in advance, feel free to add more, if you need them.

In [23]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from tensorflow.keras import layers
from tensorflow.keras import regularizers
    
print('Using TensorFlow version: %s' % tf.__version__)

Using TensorFlow version: 2.12.0


In [3]:
!pip install -q git+https://github.com/tensorflow/docs
    
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

## Data loading
Load here your data from your ML project. You can use either `pandas` or `numpy` to format your data. 

In [64]:
# Load data

dfc = pd.read_csv("/Users/linn/Desktop/neuefische/ds-artificial-neural-networks/data/client_train.csv")
dfi = pd.read_csv("/Users/linn/Desktop/neuefische/ds-artificial-neural-networks/data/ML_project.csv")

In [65]:
# we check the values for certain columns and decider, wether we transform them to categorical or not.

dfi.tarif_type= dfi.tarif_type.astype('category')
dfi.counter_type = dfi.counter_type.astype('category')
dfi.counter_code = dfi.counter_code.astype('category')
dfi.reading_remarque = dfi.reading_remarque.astype('category')

# some columns can be reduced to int8, 16 or 32 to save memory.
dfi.counter_coefficient = dfi.counter_coefficient.astype('int8')

dfi.months_number = dfi.months_number.astype('int16')

dfi.consommation_level_4 = dfi.consommation_level_4.astype('int32')
dfi.consommation_level_3 = dfi.consommation_level_3.astype('int32')
dfi.consommation_level_2 = dfi.consommation_level_2.astype('int32')
dfi.consommation_level_1 = dfi.consommation_level_1.astype('int32')

# dfi.counter_statue.value_counts()
dfi.counter_statue = dfi.counter_statue.astype('category')

In [66]:
dfi = dfi.query('avg_month_use > 100 ')

Take care of the effects of the imbalanced dataset by downsampling

In [81]:
#want to drop counter number as well

list_of_targets = dfc.query('target == 1').client_id.tolist()
y = pd.Series(dfi.client_id.isin(list_of_targets))

dfi['y'] = y

n_positive = dfi.query('y == 1').shape[0]

dfi_downsampled = dfi.query('y == 0').sample(n=n_positive)

X = pd.concat([dfi_downsampled, dfi.query('y == 1')], axis=0)

X.drop(['client_id', 'counter_number', 'Unnamed: 0'], axis=1, inplace=True)
y = X.pop('y')

In [67]:
# define target and feature (drop here the target and also get rid of the counter number)

# X = dfi.drop(['client_id', 'counter_number', 'Unnamed: 0'], axis=1) #want to drop counter number as well

# list_of_targets = dfc.query('target == 1').client_id.tolist()
# y = pd.Series(dfi.client_id.isin(list_of_targets))

In [82]:
X.columns

Index(['tarif_type', 'counter_statue', 'counter_code', 'reading_remarque',
       'counter_coefficient', 'consommation_level_1', 'consommation_level_2',
       'consommation_level_3', 'consommation_level_4', 'old_index',
       'new_index', 'months_number', 'counter_type', 'avg_month_use'],
      dtype='object')

In [ ]:
# categorical features 'tarif_type' 'counter_code' 'reading_remarque' 'counter_statue'

In [84]:
#Create dummy variables for the categorical features
cat_features = ['tarif_type', 'counter_code', 'reading_remarque', 'counter_statue']

X_cat = pd.get_dummies(X[['tarif_type', 'counter_code', 'reading_remarque', 'counter_statue']], prefix= cat_features, prefix_sep='_', dtype='uint8')
X_cat.head()

,tarif_type_8,tarif_type_9,tarif_type_10,tarif_type_11,tarif_type_12,tarif_type_13,tarif_type_14,tarif_type_15,tarif_type_18,tarif_type_21,...,reading_remarque_7,reading_remarque_8,reading_remarque_9,counter_statue_0,counter_statue_1,counter_statue_2,counter_statue_3,counter_statue_4,counter_statue_5,counter_statue_6
358601,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3124070,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3094533,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3548605,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4312300,0,0,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0


In [85]:
# Concatenate both together 

X = pd.concat([X.drop(cat_features, axis = 1), X_cat], axis = 1)
X.head()

,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type,avg_month_use,...,reading_remarque_7,reading_remarque_8,reading_remarque_9,counter_statue_0,counter_statue_1,counter_statue_2,counter_statue_3,counter_statue_4,counter_statue_5,counter_statue_6
358601,1,861,0,0,0,30919,31780,4,0,215.25,...,0,0,0,1,0,0,0,0,0,0
3124070,1,646,0,0,0,25300,25946,4,0,161.50,...,0,0,0,1,0,0,0,0,0,0
3094533,1,746,0,0,0,30446,31192,4,0,186.50,...,0,0,0,1,0,0,0,0,0,0
3548605,1,863,0,0,0,11652,12515,4,0,215.75,...,0,0,0,1,0,0,0,0,0,0
4312300,1,601,0,0,0,70913,71514,4,0,150.25,...,0,0,1,1,0,0,0,0,0,0


In [86]:
# Split the data

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [87]:
# Scale numerical values
col_scale = ['counter_coefficient', 'consommation_level_1', 'consommation_level_2', 'consommation_level_3', 'consommation_level_4', 'months_number', 'old_index', 'new_index', 'avg_month_use']

scaler = MinMaxScaler()
X_train[col_scale] = scaler.fit_transform(X_train[col_scale])
X_test[col_scale] = scaler.transform(X_test[col_scale])

## Training
For training you need a train/val split (hopefully you did a train/test split before (and you should use the same as in your ML project to make results comparable). 

In [30]:
# Train validation split  by splitting furtherthe X_train data
 # do not do it like this! The fit function has a validation split

# X_train_train = X_train.sample(frac=0.8, random_state=42)
# X_train_val = X_train.drop(X_train.index)

In [89]:
#N_VAL = len()
N_TRAIN = len(X_train)
BATCH_SIZE = 32
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE
EPOCHS = 200

### Build, compile and fit your model
To become fast at retraining your (different) models it is good practice to define a function that gets fed by a model, its name, an optimizer to use and the number of epochs you want the model to be trained. 

If your model trains for many epochs you will receive a lot of logging from TensorFlow. To reduce the logging noise you can use a callback (provided by the `tensorflow_docs` module we installed and imported for you) named `EpochDots()` that simply prints a `.` for each epoch and a full set of metrics after a number of epochs have been trained. 

If you want to produce logs for using TensorBoard you also need to include the `callbacks.TensorBoard()`.

In [ ]:
# Clearing previous logs
!rm -rf my_logs/

In [38]:
import os
import time

In [35]:
# Define path for new directory 
root_logdir = os.path.join(os.curdir, "my_logs")

In [39]:
# Define function for creating a new folder for each run
def get_run_logdir():
    run_id = time.strftime('run_%d_%m_%Y-%H_%M_%S')
    return os.path.join(root_logdir, run_id)

In [40]:
run_logdir = get_run_logdir()

In [41]:
# Create the callbacks function
def get_callbacks(name):
    return tf.keras.callbacks.TensorBoard(run_logdir+name, histogram_freq=1)

# return [list of your callbacks]

You can implement your callbacks in the `model.fit()` method below.

In [42]:
def model_compile_and_fit(model, optimizer=None ,verbose=0):
    # Get optimizer
    # model.compile
    model.compile(optimizer=optimizer,
                  loss='mae',
                  metrics=['mse'])
    # model.fit
    model.fit(X_train,
                        y_train,
                        validation_split=0.2,
                        verbose=verbose,
                        steps_per_epoch=STEPS_PER_EPOCH,
                        epochs=EPOCHS)
    return model
    
    # return results

#### Build your model
You can build your model by using `tf.keras.Sequential()` that helps you to sequentially define your different layers from input to output. 

In [78]:
#your_model = tf.keras.Sequential([
#    layers.Dense(),
#    layers.Dense()
#])    

Bane_of_frauds = tf.keras.Sequential([
      tf.keras.layers.Dense(32,kernel_initializer = 'uniform', activation='relu', input_dim = 79),
      tf.keras.layers.Dense(32,kernel_initializer = 'uniform', activation='relu'),
      tf.keras.layers.Dense(1,kernel_initializer = 'uniform')])

#### Train your model
Train your model by using your `model_compile_and_fit()` function you defined above.

In [59]:
# Model 1 was run with N_train = 1000 and only 5 epochs
optimizer_model = tf.optimizers.Adam(learning_rate= 0.1)
your_history = model_compile_and_fit(Bane_of_frauds, optimizer = optimizer_model, verbose = 1)

Epoch 1/5
31/31 [==============================] - 24s 778ms/step - loss: 859.8848 - mse: 1173132.1250 - val_loss: 408.8295 - val_mse: 195839.7656
Epoch 2/5
31/31 [==============================] - 24s 797ms/step - loss: 319.7281 - mse: 162003.1719 - val_loss: 271.1839 - val_mse: 86024.1562
Epoch 3/5
31/31 [==============================] - 22s 744ms/step - loss: 214.4667 - mse: 88363.5547 - val_loss: 162.1869 - val_mse: 30749.4395
Epoch 4/5
31/31 [==============================] - 24s 796ms/step - loss: 217.7237 - mse: 88266.7656 - val_loss: 196.5713 - val_mse: 45306.1016
Epoch 5/5
31/31 [==============================] - 24s 814ms/step - loss: 210.8453 - mse: 87295.7578 - val_loss: 192.3194 - val_mse: 43249.3242


In [61]:
# Model 2 was run with N_train = 10000 and only 5 epochs
optimizer_model = tf.optimizers.Adam(learning_rate= 0.1)
your_history = model_compile_and_fit(Bane_of_frauds, optimizer = optimizer_model, verbose = 1)

Epoch 1/5
312/312 [==============================] - 26s 83ms/step - loss: 330.4006 - mse: 233269.4375 - val_loss: 142.9357 - val_mse: 23894.4785
Epoch 2/5
312/312 [==============================] - 25s 81ms/step - loss: 300.0847 - mse: 174839.1406 - val_loss: 305.2665 - val_mse: 109067.0938
Epoch 3/5
312/312 [==============================] - 26s 83ms/step - loss: 350.5782 - mse: 181033.9219 - val_loss: 24.4779 - val_mse: 698.2937
Epoch 4/5
312/312 [==============================] - 25s 80ms/step - loss: 294.9922 - mse: 147976.0469 - val_loss: 213.3633 - val_mse: 53322.5781
Epoch 5/5
312/312 [==============================] - 26s 83ms/step - loss: 342.4513 - mse: 169848.5312 - val_loss: 5.9814 - val_mse: 41.6683


In [79]:
# Model 3 was run with now dummy variables for the cat features and N_train = 10000 and only 5 epochs
optimizer_model = tf.optimizers.Adam(learning_rate= 0.1)
your_history = model_compile_and_fit(Bane_of_frauds, optimizer = optimizer_model, verbose = 1)

Epoch 1/5
312/312 [==============================] - 26s 82ms/step - loss: 11.1855 - mse: 252.2151 - val_loss: 1.4810 - val_mse: 4.4714
Epoch 2/5
312/312 [==============================] - 25s 80ms/step - loss: 4.6224 - mse: 34.2591 - val_loss: 2.9694 - val_mse: 9.1430
Epoch 3/5
312/312 [==============================] - 25s 80ms/step - loss: 3.4819 - mse: 13.4244 - val_loss: 4.8004 - val_mse: 23.8278
Epoch 4/5
312/312 [==============================] - 25s 79ms/step - loss: 3.3469 - mse: 13.2352 - val_loss: 3.3402 - val_mse: 11.7080
Epoch 5/5
312/312 [==============================] - 24s 77ms/step - loss: 3.2806 - mse: 11.6271 - val_loss: 2.5840 - val_mse: 7.0082


In [90]:
# Model 4 was run with now dummy variables for the cat features a balanced dataset (len of double of only y=1 entries) with N_train = len(X_train) epochs = 200
optimizer_model = tf.optimizers.Adam(learning_rate= 0.1)
your_history = model_compile_and_fit(Bane_of_frauds, optimizer = optimizer_model, verbose = 1)

Epoch 1/200
7060/7060 [==============================] - 42s 6ms/step - loss: 14.6466 - mse: 363.7617 - val_loss: 2.3552 - val_mse: 5.8498
Epoch 2/200
7060/7060 [==============================] - 36s 5ms/step - loss: 14.8148 - mse: 370.4562 - val_loss: 5.9949 - val_mse: 36.4533
Epoch 3/200
7060/7060 [==============================] - 36s 5ms/step - loss: 15.8403 - mse: 425.7485 - val_loss: 9.6269 - val_mse: 93.2448
Epoch 4/200
7060/7060 [==============================] - 36s 5ms/step - loss: 14.9496 - mse: 386.9802 - val_loss: 8.4243 - val_mse: 72.6720
Epoch 5/200
5649/7060 [=======================>......] - ETA: 6s - loss: 14.8083 - mse: 363.0542

KeyboardInterrupt: 

#### Evaluate your model training
TensorFlow offers now (this was more cumbersome before) a simple history plotter that you can use to plot training histories and see how the model performed on training and validation data set.

In [ ]:
# Plot model 3 
history_plotter = tfdocs.plots.HistoryPlotter(metric = 'your_metric', smoothing_std=10)
history_plotter.plot(your_history)

## Model tuning
You might have no luck with your first model (most surely you did not). In this section you will apply methods you know to tune your model's performance. An obvious way of course is to change your model's architecture (removing or adding layers or layer dimensions, changing activation functions). 

However, after this you might still be able to detect some overfitting and there are some more methods you can apply to improve your neural network. Some of them are regularization, learning rate decay, early stopping, or dropout. 

If you want to add regularization you can apply directly layer-wise L2- or L1-regularization by using a layer's `kernel_regularization` argument and an appropriate regularizer from the [`tensorflow.keras.regularizers`](https://www.tensorflow.org/api_docs/python/tf/keras/regularizers) module we imported for you.  

__Optimizer schedules__<br>
Quite often your optimizer does not run efficiently through the loss function surface. Remember that theory ensures a convergence of mini-batch SGD if and only if the learing rate decreases sufficiently fast. A way to apply this to your model training is to use a learning rate scheduler (learning rate decay) that reduces the learning rate over the number of update steps. The [`tf.keras.optimizers.schedules`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules) module offers you some approaches to do that. 

Note that to apply this to your `model_compile_and_fit()` function you defined above you need to implement the learning rate schedule either in there or with a helper function that your function calls inside. 

If you want to visualize different schedulers you can define them and call them on a range of values and plot them in a line plot. 

__Early stopping__<br>
Earyl stopping is a procedure that enables you to stop your training earlier than defined by your `max_epochs` argument. It is used in practices to 
1. determine the optimal parameter vector by monitoring the validation error closely (if it rises again too much stuck with the best parameters found until then) and
2. to save expensive resources (either in terms of monetary costs or ecological costs).

To implement early stopping in TensorFlow the `tf.keras` module offers you a `callback` named [`tf.keras.callback.EarlyStopping()`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) that monitors for you a certain metric (it makes sense here to use a validation metric) and to stop training after a certain number of epochs with no improvement or by defining a certain `min_delta` that defines a minimum value of improvement - if below the callback stops your training. 

You can add this callback simply to the callbacks defined in your `get_callbacks()` function you defined above.

__Dropout__<br>
Dropout was one of the important developments in regularization for neural networks. It was developed by Geoffrey Hinton and his team at Toronto University. 

Dropout can be applied to each layer in your network and is implemented in `tf.keras` by an own layer named `Dropout()` awaiting a dropout rate set by you. So to introduce dropout you have to rework your model design.  

Make use of your knowledge and apply tuning techniques to improve your network. 

In [ ]:
#===========#
# YOUR CODE #
#===========#

## Model ensembling
You have learned that models can be ensembled. What is possible in `scikit-learn` is also possible in TensorFlow, just a little different as it is relying on its computation graph. However, any model is callable like a `layer` by invoking it on either an `Input` or on the output of another layer. Furthermore, you can also stack outputs together.

To produce an ensemble you can define a couple of models, than use their predictions as inputs for another model and produce a final output (using `keras.Model(input, output)`). But you can also start simple and use the mean predictions over all models and then compute the `argmax()` to assign them to a class in classification (via using `layers.average([model1_preds,model2_preds,...])`). You will be surprised how well this works. 

Now implement your own ensemble to improve your work even a little more and to have something more to polish up your ML project on `GitHub` ;) 

In [ ]:
#===========#
# YOUR CODE #
#===========#